<a href="https://colab.research.google.com/github/vvshyer/tensorflow2.0_learning/blob/master/tf1_dense_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==1.13.1

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

1.13.1
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [3]:
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [4]:
print(np.max(x_train), np.min(x_train))

255 0


In [0]:
# x = (x-u)/std
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1, 28* 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1, 28* 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1, 28* 28)

In [9]:
print(np.max(x_train_scaled), np.min(x_train_scaled))

2.0231433 -0.8105136


In [10]:
hidden_units = [100, 100]
class_num = 10

x = tf.placeholder(tf.float32, [None, 28* 28])
y = tf.placeholder(tf.int64, [None])

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                           hidden_unit, 
                                           activation=tf.nn.relu)
logits = tf.layers.dense(input_for_next_layer, class_num)
# last_hidden_output * W(logits) -> softmax -> prob
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels = y, logits = logits)

prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Instructions for updating:
Use tf.cast instead.


In [11]:
print(x)
print(y)

Tensor("Placeholder_2:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_3:0", shape=(?,), dtype=int64)


In [13]:
# session
init = tf.global_variables_initializer()
batch_size = 20
epochs = 10
train_steps_per_epoch = x_train.shape[0] // batch_size
valid_steps = x_valid.shape[0] // batch_size

def eval_with_sess(sess, x, y, accuracy, images, labels, batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[step * batch_size : (step+1)*batch_size]
        batch_label = labels[step*batch_size : (step+1)*batch_size]
        accuracy_val = sess.run(accuracy,
                                feed_dict = {
                                    x: batch_data,
                                    y: batch_label
                                })
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)
    
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            batch_data = x_train_scaled[
                step * batch_size : (step+1) * batch_size]
            batch_label = y_train[
                step * batch_size : (step+1) * batch_size]
            loss_val, accuracy_val, _ = sess.run(
                [loss, accuracy, train_op],
                feed_dict = {
                    x: batch_data,
                    y: batch_label
                })
            print('\r[Train] epoch : %d, step: %d, loss: %3.5f, accuracy: %2.2f' %
                  (epoch, step, loss_val, accuracy_val), end="")
        valid_accuracy = eval_with_sess(sess, x, y, accuracy, x_valid_scaled, y_valid, batch_size)
        print('\t[Valid] acc: %2.2f' % valid_accuracy)

[Train] epoch : 0, step: 2749, loss: 0.33170, accuracy: 0.80	[Valid] acc: 0.86
[Train] epoch : 1, step: 2749, loss: 0.29312, accuracy: 0.80	[Valid] acc: 0.86
[Train] epoch : 2, step: 2749, loss: 0.24072, accuracy: 0.85	[Valid] acc: 0.87
[Train] epoch : 3, step: 2749, loss: 0.14616, accuracy: 0.90	[Valid] acc: 0.87
[Train] epoch : 4, step: 2749, loss: 0.16842, accuracy: 0.90	[Valid] acc: 0.88
[Train] epoch : 5, step: 2749, loss: 0.14214, accuracy: 0.95	[Valid] acc: 0.88
[Train] epoch : 6, step: 2749, loss: 0.08052, accuracy: 1.00	[Valid] acc: 0.88
[Train] epoch : 7, step: 2749, loss: 0.14934, accuracy: 0.95	[Valid] acc: 0.88
[Train] epoch : 8, step: 2749, loss: 0.12801, accuracy: 0.95	[Valid] acc: 0.88
[Train] epoch : 9, step: 2749, loss: 0.16560, accuracy: 0.95	[Valid] acc: 0.88
